# Week 2. Day 2. Exercises from Chapter 5 of FSStDS. 
## Fundamentals of Social Data Science. MT 2022

Within your study pod discuss the following questions. Please submit an individual assignment by 12:30pm Wednesday, October 18, 2022 on Canvas. 

In [1]:
import pandas as pd 

# Exercise 1. Twitter merging 

I have provided two tables: `dalle2_oct18_2022_tweets.csv` and `dalle2_oct18_2022_users.csv`. You can see how these tweets were collected in the Appendix to this assignment. It's a simple pull of only 100 tweets. To continue this pull would require paging (another day). For now, let's focus on merging. Please merge these two tables. 

Some tips: 
- Ensure that you keep all the tweets.
- Ensure that the names which might overlap (hint...`id`) are given descriptive suffixes.
- Your resulting df should still have 100 rows. 

In [138]:
# Exercise 1 below here 

tweet_df = pd.DataFrame([])
users_df = pd.DataFrame([])
merge_df = pd.DataFrame([])

print(len(tweet_df),len(users_df),len(merge_df))
# Should be 100 79 100

0 0 0


# Exercise 2. Twitter analytics 

Split the data into two groups: 
- Those with more than 1000 followers and those with less
- Compare the two groups. Which group has more tweets and _proportionately_ more @mentions in their tweets.
    
> Note: Getting the @mentions can be done cheap and easy (search for @ symbol) or more robust and with a little more difficulty (look in the entities.mentions column and wrangle the dictionary)

In [140]:
# Exercise 2. Answer below here

len_over1k = ...
len_under1k = ...
over1k_nummention = ...
under1k_nummention = ...

print(f"The percentage of tweets from those with over 1k followers that have mentions is  {over1k_nummention / len_over1k:0.1%}",
      f"The percentage of tweets from those with under 1k followers that have mentions is  {under1k_nummention / len_under1k:0.1%}")

# Should be 29 for over1k and 71 for under1k
# And therefore should be 24.1% and 11.3% respectively.

TypeError: unsupported operand type(s) for /: 'ellipsis' and 'ellipsis'

# Exercise 3. Grouping the data

Group the data by Author and build a table that reports the max, min, and average for both  `public_metrics.retweet_count` and `public_metrics.like_count`. 

In [105]:
# Exercise 3. Answer below here

# Should be one line

# Exercise 4. Twitter Reshaping

Create a long `DataFrame` of tweet_ids, author_ids, and hash_tags. That is, one row per hashtag rather than one per tweet. Report the length of this `DataFrame` and the `value_counts()` of the top 10 hashtags.

In [76]:
def get_taglist(tag_list_full):
    
    if type(tag_list_full) == list:
        return [x['tag'] for x in tag_list_full]
    else:
        return None
    
new_df['tags'] = new_df['entities.hashtags'].map(get_taglist)

In [88]:
long_df = new_df[['id_x','tags','username']].explode('tags')

print(f"The length of the exploded data by hashtag is {len(long_df)}")
print("The top ten hashtags are as follows:",
       long_df['tags'].value_counts()[:10],
      sep="\n")

The length of the exploded data by hashtag is 605
The top ten hashtags are as follows:
dalle2             77
aiart              26
dalle              24
ai                 23
stablediffusion    22
digitalart         20
midjourney         20
aiartist           15
aiartcommunity     14
AIart              14
Name: tags, dtype: int64


# Appendix: How I pre-processed the data (See Chapter 7) 




In [9]:
import os
import requests
import dotenv

ENV_PATH = f"..{os.sep}.env"
dotenv.load_dotenv(ENV_PATH) # This will refresh the environment variables
print(len(os.environ.get('TWITTER_BEARER_TOKEN')))

In [93]:
URL = "https://api.twitter.com/2/tweets/search/all"

BEARER = os.environ["TWITTER_BEARER_TOKEN"]
headers = {"Authorization": f"Bearer {BEARER}"}

QUERY = "(dalle2) -is:retweet"
MAX_RESULTS = 100 

params={"query": QUERY,
        "max_results":MAX_RESULTS}

params['expansions'] = "author_id,geo.place_id"
params['tweet.fields'] = "entities,public_metrics"
params['user.fields'] = "id,username,name,description,public_metrics"
params['place.fields'] = "id,country,country_code,full_name"

response = requests.get(URL, headers=headers, params=params)

assert response.status_code == 200, \
    f"Code {response.status_code}. See error: {response.json()}"

tweets = response.json()
print(tweets.keys())

dict_keys(['data', 'includes', 'meta'])


In [94]:
import json 

json.dump(tweets['data'], 
          open("dalle2_oct18_2022_tweets.json",'w')) 

json.dump(tweets['includes']['users'],
          open("dalle2_oct18_2022_users.json",'w')) 